<a href="https://colab.research.google.com/github/kulkarnisatishp/genai-course/blob/main/multimodel_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from google.colab import userdata
openai_api_key = userdata.get('openai_key')

In [21]:
%cd /content/drive/MyDrive/GenAI/RAG/Multimodal RAG

/content/drive/MyDrive/GenAI/RAG/Multimodal RAG


In [22]:
video_path = "decision-making-course.mp4"

In [23]:
!pip install -q pydub
!apt-get install -q ffmpeg

Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [24]:
import os
import subprocess
from pydub import AudioSegment

In [25]:
audio_output_path = "audios/output.mp3"

In [28]:
command = [
    'ffmpeg', '-y', '-i', video_path, '-vn', '-acodec', 'libmp3lame', audio_output_path
]

In [29]:
subprocess.run(command, check=True)

CompletedProcess(args=['ffmpeg', '-y', '-i', 'decision-making-course.mp4', '-vn', '-acodec', 'libmp3lame', 'audios/output.mp3'], returncode=0)

In [31]:
bitrate = "32k"
compressed_audio_path = "audios/compressed_audio.mp3"

In [32]:
command = [
    'ffmpeg', '-y', '-i', audio_output_path, '-ab', bitrate, compressed_audio_path
]
subprocess.run(command, check=True)

CompletedProcess(args=['ffmpeg', '-y', '-i', 'audios/output.mp3', '-ab', '32k', 'audios/compressed_audio.mp3'], returncode=0)

In [33]:
!pip install -q openai

In [34]:
from openai import OpenAI


In [35]:
client = OpenAI(api_key=openai_api_key)

In [36]:
with open(compressed_audio_path, "rb") as file:
    transcript = client.audio.transcriptions.create(
        model="whisper-1",
        file=file
    )

In [37]:
transcription_path = "transcripts/transcript.txt"

In [38]:
with open(transcription_path, "w") as file:
    file.write(transcript.text)


In [39]:
!pip install -q moviepy

In [40]:
from moviepy.editor import VideoFileClip

  if event.key is 'enter':



In [42]:
output_folder = "frames"
video = VideoFileClip(video_path)

In [43]:
frame_paths = []
interval = 10

for t in range(0, int(video.duration), interval):
  frame_path  = os.path.join(output_folder, f"frame_{t:04d}.png")
  video.save_frame(frame_path, t=t)
  frame_paths.append(frame_path)


In [46]:
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
import torch
import numpy as np

In [47]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [50]:
transcript_text = transcript.text

In [54]:
tokens = tokenizer(transcript_text, return_tensors="pt", padding=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (9902 > 77). Running this sequence through the model will result in indexing errors


In [55]:
tokens = tokens['input_ids'][0]
len(tokens)

9902

In [56]:
max_tokens = 77
transctiption_chunks = []

for i in range(0, len(tokens), max_tokens):
  chunk = tokens[i:i+max_tokens]
  transctiption_chunks.append(chunk)

len(transctiption_chunks)

129

In [61]:
text_embeddings =[]
for chunk in transctiption_chunks:
  inputs = {"input_ids": transctiption_chunks[0].unsqueeze(0)}
  inputs
  with torch.no_grad():
    text_embedding = model.get_text_features(**inputs)
    text_embeddings.append(text_embedding.cpu().numpy().flatten())

text_embedding_np = np.array(text_embeddings)
text_embedding_np.shape

(129, 512)